In [1]:
pip install langchain langchain-openai pydantic

Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field
from langchain_core.output_parsers import PydanticOutputParser

# 1. Define structured output
class Task(BaseModel):
    task_name: str = Field(description="Name of the task.")
    estimated_days: int = Field(description="Estimated days to complete the task.")
    team_member: str = Field(description="Assigned team member's name.")

class ProjectPlan(BaseModel):
    project_description: str = Field(description="Overall description of the project.")
    tasks: list[Task] = Field(description="List of detailed tasks for the project.")
    risks: list[str] = Field(description="Potential risks associated with the project plan.")
    steps:list[str] = Field(description= "Steps to Write Better AI Prompts for Project Manager.")
    phase_details: list[str] = Field(description="In Depth details for this project phase.")
# 2. Initialize LLM
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.2)

# 3. Define prompt
parser = PydanticOutputParser(pydantic_object=ProjectPlan)

prompt = ChatPromptTemplate.from_messages([
    ("system", """You are a world-class project manager assistant. Your goal is to create a structured project plan in JSON format. {format_instructions}
    Requirements:
        - Each section must be approximately 1000 words.
        - Ensure all dates are in the year 2026.
        - Ensure all details are in-depth, comprehensive, and professional.
    {format_instructions}"""),
    ("human", "Create a project plan for the following project: {project_goal}")
]).partial(format_instructions=parser.get_format_instructions())

# 4. Build chain
planning_chain = prompt | llm | parser

# 5. Run chain
project_goal_input = input("Enter a subject: ")

#project_goal_input = "Develop a basic e-commerce website using Python and Django."
#print(f"Generating project plan for: '{project_goal_input}'...\n")

response = planning_chain.invoke({"project_goal": project_goal_input})

# 6. Process output
print("--- Generated Project Plan ---")
print(f"Project Description: {response.project_description}\n")

print("Tasks:")
for i, task in enumerate(response.tasks):
    print(f"* {i+1}. {task.task_name} ({task.estimated_days} days) - Assigned to: {task.team_member}")

print("\nRisks:")
for i, risk in enumerate(response.risks):
    print(f"* {i+1}. {risk}")

print("\nSteps for Project Manager:")
for i, step in enumerate(response.steps):
    print(f"* {i+1}. {step}")
    
print("Project Phase:")
for i, phase in enumerate(response.phase_details):
    print(f"* {i+1}. {phase}")




Enter a subject:  Develop a basic e-commerce website using Python and Django.


--- Generated Project Plan ---
Project Description: The project aims to develop a basic e-commerce website using Python and Django. The website will serve as a platform for users to browse products, add items to their cart, and make purchases online. The website will also include features such as user authentication, product search functionality, and a simple checkout process. The goal is to create a functional e-commerce site that showcases the capabilities of Python and Django in web development.

Tasks:
* 1. Project Kickoff Meeting (1 days) - Assigned to: Project Manager
* 2. Gather Requirements (3 days) - Assigned to: Business Analyst
* 3. Design Database Schema (2 days) - Assigned to: Database Architect
* 4. Set Up Django Environment (1 days) - Assigned to: DevOps Engineer
* 5. Create Models for Products and Users (2 days) - Assigned to: Backend Developer
* 6. Implement User Authentication (3 days) - Assigned to: Backend Developer
* 7. Develop Product Listing Page (4 days) - Assig